In [1]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy
import data_generation as dg
import time

In [3]:
graph, random_solution, total_edge_length = dg.read_instance('data\data8.txt')
num_nodes = len(graph)

In [4]:
sols = []
values = []
iters = []
times = []

<span style="font-size: larger;">Classic local search</span>

In [5]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    index1, index2 = random.sample(range(len(solution)), 2)
    
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [6]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [7]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    #we will be using this in a modified local search when regular methods get stuck in hopes of finding
    #a better solution close to the current one
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    if k == -1:
        return sorted(new_solution)

    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [8]:
def make_change_scramble(graph, solution):
    #another change function, as the name suggest, this one will scramble a section of the solution
    #this wouldn't be the method we use if we only tested one version, but since there are plenty of change functions and combinations,
    #this one might prove most useful in larger dimension
    new_solution = deepcopy(solution)
    
    start, end = sorted(random.sample(range(len(solution)), 2))

    subset = solution[start:end+1]

    random.shuffle(subset)

    new_solution[start:end+1] = subset

    return new_solution

In [9]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [10]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 2, 0, 6, 7, 4, 3, 1] 21 23


In [11]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[1, 3, 6, 4, 7, 2, 0, 5] 21 216


In [12]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 0, 2, 6, 7, 4, 3, 1] 21 98


In [13]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] == best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    if values[i] < best:
        best = values[i]
        best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,8,make_change_swap,21,0.01
1,8,make_change_inverse,21,0.02
2,8,make_change_scramble,21,0.02


In [14]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_swap 21 0.01 23


,Dim,Method,Value,Time,Best_Iter
0,8,make_change_swap,21,0.01,23


In [15]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

#check if first write to the specified row
if not row_to_update.empty:
    #update specific columns in the located row
    df.loc[row_to_update.index, 'local_search'] = values[best_i]
    df.loc[row_to_update.index, 'local_search_time'] = times[best_i]

    #save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'local_search': values[best_i], 'local_search_time': times[best_i]}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [16]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [21.0, 0.02, 112.33]


,Dim,Value,Time,Best_Iter
0,8,21.0,0.02,112.33


In [17]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    df.loc[row_to_update.index, 'local_search'] = average[0]
    df.loc[row_to_update.index, 'local_search_time'] = average[1]


    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'local_search': average[0], 'local_search_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Local search with next permutation help</span>

In [18]:
sols = []
values = []
iters = []
times = []

def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = dg.calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i

In [19]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 5, 2, 0, 4, 7, 3, 1] 21 10


In [20]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[1, 3, 7, 4, 0, 2, 5, 6] 21 108


In [21]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[1, 3, 7, 4, 6, 0, 2, 5] 21 22


In [22]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__ + '_with_permutation'
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] == best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    if values[i] < best:
        best = values[i]
        best_i = i    

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,8,make_change_swap_with_permutation,21,0.09
1,8,make_change_inverse_with_permutation,21,0.09
2,8,make_change_scramble_with_permutation,21,0.09


In [23]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__ + '_with_permutation', 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_scramble 21 0.09 22


,Dim,Method,Value,Time,Best_Iter
0,8,make_change_scramble_with_permutation,21,0.09,22


In [24]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    df.loc[row_to_update.index, 'local_search_perm'] = values[best_i]
    df.loc[row_to_update.index, 'local_search_perm_time'] = times[best_i]

    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'local_search_perm': values[best_i], 'local_search_perm_time': times[best_i]}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [25]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [21.0, 0.09, 46.67]


,Dim,Value,Time,Best_Iter
0,8,21.0,0.09,46.67


In [26]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    df.loc[row_to_update.index, 'local_search_perm'] = average[0]
    df.loc[row_to_update.index, 'local_search_perm_time'] = average[1]

    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'local_search_perm': average[0], 'local_search_perm_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Simulated annealing</span>

In [27]:
sols = []
values = []
iters = []
times = []

def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        #print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        #elif random.random() < 1 / (i**0.5):
        elif random.random() < 1 / i:
            #print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [28]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 2, 0, 6, 4, 7, 3, 1] 21 87


In [29]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 2, 0, 7, 6, 4, 3, 1] 21 101


In [30]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 0, 2, 6, 4, 7, 3, 1] 21 28


In [31]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': 'simulated_annealing' + ', '  + method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] == best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    if values[i] < best:
        best = values[i]
        best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,8,"simulated_annealing, make_change_swap",21,0.01
1,8,"simulated_annealing, make_change_inverse",21,0.01
2,8,"simulated_annealing, make_change_scramble",21,0.02


In [32]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  'simulated_annealing' + ', '  + solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_inverse 21 0.01 101


,Dim,Method,Value,Time,Best_Iter
0,8,"simulated_annealing, make_change_inverse",21,0.01,101


In [33]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'sim_annealing'] = values[best_i]
    df.loc[row_to_update.index, 'sim_annealing_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'sim_annealing': values[best_i], 'sim_annealing_time': times[best_i]}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [34]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [21.0, 0.01, 72.0]


,Dim,Value,Time,Best_Iter
0,8,21.0,0.01,72.0


In [35]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'sim_annealing'] = average[0]
    df.loc[row_to_update.index, 'sim_annealing_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'sim_annealing_perm': average[0], 'sim_annealing_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Variable neigborhood search</span>

In [36]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [37]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [38]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            #print('vns: ', solution, value, i)
            #diversification
            new_solution = shaking_func(graph, solution, k)   

            new_value = dg.calculate_total_edge_length(graph, new_solution)

            #print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)

            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i

                value = new_value

                solution = deepcopy(new_solution)
                
    return solution, value, best_i

In [39]:
#too many combos for VNS, we will iterrate
#this soulution SHOULD NOT be used regularly, as the cell execution time will be MUCH longer than the sum of it's parts for some reason (notebook problem?)
#for example, for a certain n, if we split this into seperate cells the sum of exec time would be 3mins, this way it's around 5-6mins
#hopefully this doesn't seem to affect individual duration calculations...
n = len(graph)

oc = False

shakings = [shaking_swap, shaking_inverse]
changes_funcs = [make_change_swap, make_change_inverse, make_change_scramble]
search_funcs = [local_search, local_search_with_permutation]

number_of_combinations = 0

sols = []
values =[]
times = []

methods = []

results = []

best = float('inf')
best_i = -1

average = [0,0]

k_min = int(0.1 * num_nodes)
k_max = int(0.3 * num_nodes)
for shake in shakings:
    for change in changes_funcs:
        for search in search_funcs:
            if shake.__name__ == 'shaking_inverse':
                k_min = int(0.3 * num_nodes)
                k_max = int(0.6 * num_nodes)

            start = time.time()    
            sol, value, iter = vns(graph, 
                                   random_solution, 
                                   total_edge_length, 
                                   500, 
                                   change_func=change, 
                                   shaking_func=shake, 
                                   local_search_func = search, 
                                   k_min=k_min,
                                   k_max=k_max, 
                                   move_prob=0.4)
            end = time.time()
            duration = float("{:.2f}".format(end - start))
            
            print(shake.__name__, change.__name__, search.__name__, ' : ', sol, value, iter, '\n')
            
            k_min = int(0.1 * num_nodes)
            k_max = int(0.3 * num_nodes)

            sols.append(sol)
            values.append(value)
            iters.append(iter)
            times.append(duration)

            methods.append(str(shake.__name__ + ', ' + change.__name__ + ', ' + search.__name__))

            results.append({'Dim': num_nodes, 'Method': methods[number_of_combinations], 'Value': abs(value), 'Time': duration})

            if value == best:
                if duration <= times[best_i]:    
                    best = value
                    best_i = number_of_combinations

            if value < best:
                best = value
                best_i = number_of_combinations            

            average[0] += value
            average[1] += duration

            number_of_combinations += 1    


shaking_swap make_change_swap local_search  :  [1, 3, 7, 4, 0, 5, 2, 6] 21 2 



shaking_swap make_change_swap local_search_with_permutation  :  [6, 2, 5, 0, 4, 7, 3, 1] 21 3 

shaking_swap make_change_inverse local_search  :  [5, 2, 0, 7, 6, 4, 3, 1] 21 4 

shaking_swap make_change_inverse local_search_with_permutation  :  [1, 3, 7, 4, 0, 2, 5, 6] 21 0 

shaking_swap make_change_scramble local_search  :  [1, 3, 7, 4, 0, 6, 2, 5] 21 0 

shaking_swap make_change_scramble local_search_with_permutation  :  [6, 2, 5, 0, 4, 7, 3, 1] 21 3 

shaking_inverse make_change_swap local_search  :  [6, 5, 2, 0, 4, 7, 3, 1] 21 1 

shaking_inverse make_change_swap local_search_with_permutation  :  [5, 2, 6, 0, 7, 4, 3, 1] 21 2 

shaking_inverse make_change_inverse local_search  :  [1, 3, 4, 7, 6, 0, 2, 5] 21 6 

shaking_inverse make_change_inverse local_search_with_permutation  :  [5, 0, 2, 6, 4, 7, 3, 1] 21 1 

shaking_inverse make_change_scramble local_search  :  [5, 2, 0, 7, 4, 6, 3, 1] 21 2 

shaking_inverse make_change_scramble local_search_with_permutation  :  [5, 6, 2, 0, 4,

In [40]:
df = pd.DataFrame(results)
display(df)
df.to_csv('comparison_tables/vns.csv', mode='a', header=not pd.io.common.file_exists('comparison_tables/vns.csv'), index=False)

,Dim,Method,Value,Time
0,8,"shaking_swap, make_change_swap, local_search",21,0.27
1,8,"shaking_swap, make_change_swap, local_search_w...",21,0.92
2,8,"shaking_swap, make_change_inverse, local_search",21,0.26
3,8,"shaking_swap, make_change_inverse, local_searc...",21,0.64
4,8,"shaking_swap, make_change_scramble, local_search",21,0.47
5,8,"shaking_swap, make_change_scramble, local_sear...",21,1.00
6,8,"shaking_inverse, make_change_swap, local_search",21,0.27
7,8,"shaking_inverse, make_change_swap, local_searc...",21,0.86
8,8,"shaking_inverse, make_change_inverse, local_se...",21,0.28
9,8,"shaking_inverse, make_change_inverse, local_se...",21,0.87


In [41]:
print('best:', methods[best_i], values[best_i], times[best_i])
df_best = pd.DataFrame({'Dim': num_nodes, 'Method':  methods[best_i], 'Value': values[best_i], 'Time': times[best_i]}, index=[0])
display(df_best)

best: shaking_swap, make_change_inverse, local_search 21 0.26


,Dim,Method,Value,Time
0,8,"shaking_swap, make_change_inverse, local_search",21,0.26


In [42]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    
    df.loc[row_to_update.index, 'vns'] = values[best_i]
    df.loc[row_to_update.index, 'vns_time'] = times[best_i]

    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'vns': value, 'vns_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [43]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': num_nodes, 'Value': average[0], 'Time': average[1]}, index=[0])
display(df_avg)

average: [21.0, 0.6]


,Dim,Value,Time
0,8,21.0,0.6


In [44]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:

    df.loc[row_to_update.index, 'vns'] = average[0]
    df.loc[row_to_update.index, 'vns_time'] = average[1]

    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'vns_perm': average[0], 'vns_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)